# CaLPA Scratch Folder

This is a scratch notebook used to test the code and functionality of the AI California Legislative Policy Analysis (CALPA) system. It is not intended for production use and may contain incomplete or experimental code. The purpose of this notebook is to facilitate the development and testing of the CALPA system, including its data processing, analysis, and visualization components. The notebook may include code snippets, comments, and notes related to the development process. Please refer to the official documentation and user guides for the CALPA system for more information on its usage and features.

In [ ]:
#%reset

## Initialization

In [226]:
# Import required libraries
import os
import time
from datetime import date
from datetime import datetime
import json
import mimetypes
import glob
import base64
import zipfile
import io
import dotenv
import requests
import pandas as pd
import feedparser
import webbrowser
from mrkdwn_analysis import MarkdownAnalyzer

In [15]:
# Load the Calpa module located in the scripts/python/calpa directory
from calpa import Calpa, LegiScan

# Load environment variables from .env file
dotenv.load_dotenv(os.path.join(os.getcwd(), '.env'))

# Instantiate the LegiScan and Calpa classes
calpa = Calpa()
legiscan = LegiScan()

# Create project metadata for the AI project
prjMetadata = calpa.projectMetadata("AI", "0")

# Create the project directories dictionary
prjDirs = calpa.projectDirectories(os.getcwd())


Project Global Settings:
- Name: California Legislative Policy Analysis
- Title: AI Legislative Policy Analysis
- Version: 1.0
- Author: Dr. Kostas Alexandridis, GISP
Data Dates
- Start Date: 2010-12-02
- End Date: 2025-04-25
- Periods: 2009-2010, 2011-2012, 2013-2014, 2015-2016, 2017-2018, 2019-2020, 2021-2022, 2023-2024, 2025-2026
Directory Global Settings:

General:
- Project (pathPrj): c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA
- Admin (pathAdmin): c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\admin
- Metadata (pathMetadata): c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\metadata
- Analysis (pathAnalysis): c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\analysis
Scripts:
- Python Calpa Module (pathScriptsCalpa): c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\calpa
- Markdown Scripts (pathScriptsMd): c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\markdown
- RIS Scripts (pathScriptsRis): c:\Users\ktale\OneDrive\Documents\GitHub\CaLPA\ris
Data:
- Main Data (pathData): c:\Users\kt

In [16]:
# Load the codebookBill pickle file from the data/lookup directory
codebookBill = pd.read_pickle(os.path.join(prjDirs["pathDataLookup"], "codebookBill.pkl"))

# Load the codebookRollCall pickle file from the data/lookup directory
codebookRollCall = pd.read_pickle(os.path.join(prjDirs["pathDataLookup"], "codebookRollCall.pkl"))

# Load the codebookBillText pickle file from the data/lookup directory
codebookBillText = pd.read_pickle(os.path.join(prjDirs["pathDataLookup"], "codebookBillText.pkl"))

# Load the codebookAmendment pickle file from the data/lookup directory
codebookAmendment = pd.read_pickle(os.path.join(prjDirs["pathDataLookup"], "codebookAmendment.pkl"))

# Load the codebookSupplement pickle file from the data/lookup directory
codebookSupplement = pd.read_pickle(os.path.join(prjDirs["pathDataLookup"], "codebookSupplement.pkl"))

# Load the codebookPerson pickle file from the data/lookup directory
codebookPerson = pd.read_pickle(os.path.join(prjDirs["pathDataLookup"], "codebookPerson.pkl"))

# Load the codebookSessionList pickle file from the data/lookup directory
codebookSessionList = pd.read_pickle(os.path.join(prjDirs["pathDataLookup"], "codebookSessionList.pkl"))

In [17]:
# Obtain the stored sessions list from JSON dictionary on disk (data/lookup directory)
sessionListStored = legiscan.getStoredData(dataType = "session")

# Obtain the stored session People list from JSON dictionary on disk (data/lookup directory)
sessionPeopleStored = legiscan.getStoredData(dataType = "people")

# Obtain the stored dataset list from JSON dictionary on disk (data/lookup directory)
datasetListStored = legiscan.getStoredData(dataType = "dataset")

# Get the stored raw master list from JSON dictionary on disk (data/lookup directory)
masterListRawStored = legiscan.getStoredData(dataType = "master", raw = True)
# Get the stored master list from JSON dictionary on disk (data/lookup directory)
masterListStored = legiscan.getStoredData(dataType = "master", raw = False)

# Get the AI monitoring list from disk (data/lookup directory)
aiBillListStored = legiscan.getStoredData(dataType = "bills", project = "AI")

# Get the AI full list of bills from dism (data/legis/json directory)
aiBills = legiscan.getStoredData(dataType = "data", project = "AI")

# Get the AI bill summries list from disk (data/lookup directory)
aiBillsSummariesStored = legiscan.getStoredData(dataType = "summaries", project = "AI")

## End of Initialization

In [5]:
test = calpa.getCaLegisLinks(billPeriod = "2013-2014", billId = "SB860")

In [7]:
test.keys()

dict_keys(['main', 'text', 'votes', 'history', 'analysis', 'todaysLaw', 'compare', 'status'])

In [6]:
# open test["main"] in a web browser
webbrowser.open(test["status"])

True

In [12]:
test.keys()

dict_keys(['main', 'text', 'votes', 'history', 'analysis', 'todaysLaw', 'compare', 'status'])

In [ ]:
testBill = aiBills["2013-2014"]["SB860"]
testBill.keys()

In [ ]:
legiscan.billCode[bType]

In [21]:
aiBills["2013-2014"]["SB860"]

{'bill_id': 581712,
 'change_hash': 'e38cd0ab53209fc7aaf182ffc8ba1f5b',
 'session_id': 993,
 'session': {'session_id': 993,
  'state_id': 5,
  'year_start': 2013,
  'year_end': 2014,
  'prefile': 0,
  'sine_die': 1,
  'prior': 1,
  'special': 0,
  'session_tag': 'Regular Session',
  'session_title': '2013-2014 Regular Session',
  'session_name': '2013-2014 Session'},
 'url': 'https://legiscan.com/CA/bill/SB860/2013',
 'state_link': 'http://www.leginfo.ca.gov/cgi-bin/postquery?bill_number=sb_860&sess=1314&house=S',
 'completed': 1,
 'status': 4,
 'status_date': '2014-06-20',
 'progress': [{'date': '2014-01-09', 'event': 1},
  {'date': '2014-01-23', 'event': 9},
  {'date': '2014-04-10', 'event': 2},
  {'date': '2014-05-08', 'event': 9},
  {'date': '2014-06-12', 'event': 9},
  {'date': '2014-06-15', 'event': 3},
  {'date': '2014-06-20', 'event': 8},
  {'date': '2014-06-20', 'event': 4}],
 'state': 'CA',
 'state_id': 5,
 'bill_number': 'SB860',
 'bill_type': 'B',
 'bill_type_id': '1',
 'bo

In [ ]:
# join a list of strings with a comma
def joinList(list):
    return ", ".join(list)

In [37]:
", ".join(aiBillsSummariesStored["2013-2014"]["SB860"]["tags"])

'artificial-intelligence, california, higher-education, budget, student-aid, cal-grant, middle-class-scholarship, community-colleges, calworks, university-of-california, california-state-university, funding, neuroscience, cal-brain, education-policy, legislation'

In [39]:
", ".join([f"#{tag}" for tag in aiBillsSummariesStored["2013-2014"]["SB860"]["tags"]])

'#artificial-intelligence, #california, #higher-education, #budget, #student-aid, #cal-grant, #middle-class-scholarship, #community-colleges, #calworks, #university-of-california, #california-state-university, #funding, #neuroscience, #cal-brain, #education-policy, #legislation'

In [ ]:
def aiBillYaml(billPeriod, billId):
    """
    Function to create a YAML file for the AI bill
    """
    # Get the bill data from the AI bills dictionary
    bill = aiBills[billPeriod][billId]
    
    thisPeriod = billPeriod
    thisBillId = billId
    thisBillCode = f"{bill['body']}{bill['bill_type']}"
    thisBillNumber = bill['bill_number'].replace(f"{thisBillCode}", "")
    thisBillAlias1 = f"{thisBillCode}-{thisBillNumber}"
    thisBillAlias2 = f"{thisBillCode} {thisBillNumber}"
    thisBillKeywords = ", ".join(aiBillsSummariesStored[thisPeriod][thisBillId]["tags"])
    thisBillHashTags = ", ".join([f"#{tag}" for tag in aiBillsSummariesStored[thisPeriod][thisBillId]["tags"]])
    thisBillSponsors = legiscan.summarizeBillSponsors(aiBills[thisPeriod][thisBillId], output="dict")
    thisBillAction = bill['history'][-1]['action']
    if thisBillAction.startswith("Chaptered"):
        chaptered = True
        chapterNo = thisBillAction.removeprefix("Chaptered by Secretary of State. Chapter ").removesuffix(".")
        chapterYear = int(chapterNo.split(", Statutes of ")[1])
        chapterNo = int(chapterNo.split(", Statutes of ")[0])
    else:
        chaptered = False
        chapterYear = None
        chapterNo = None
    
    legisLinks = calpa.getCaLegisLinks(billPeriod, billId)
    
    # Create the YAML file name
    yamlFileName = f"{billId}.yaml"
    
    # Create the YAML file path
    yamlFilePath = os.path.join(prjDirs["pathData"], "legis", "yaml", thisPeriod, yamlFileName)
    
    # Create the YAML file
    with open(yamlFilePath, 'w') as yamlFile:
        # Begin writing the YAML file
        yamlFile.write(f"---\n")
        # Aliases (vector)
        yamlFile.write(f"aliases:\n")
        yamlFile.write(f"  - {thisBillAlias1}\n")
        yamlFile.write(f"  - {thisBillAlias2}\n")
        # Type (vector)
        yamlFile.write(f"type:\n")
        yamlFile.write(f"  - {legiscan.billCode[thisBillCode]}\n")
        yamlFile.write(f"  - California Legislature Bill\n")
        yamlFile.write(f"  - AI Legislation\n")
        # Tags (vector)
        yamlFile.write(f"tags:\n")
        yamlFile.write(f"  - Zotero\n")
        yamlFile.write(f"  - california-legislature\n")
        yamlFile.write(f"  - {bill['session']['session_title'].lower().replace(' ', '-')}\n")
        for tag in aiBillsSummariesStored[thisPeriod][thisBillId]["tags"]:
            yamlFile.write(f"  - {tag}\n")
        # Keywords
        yamlFile.write(f"keywords: {thisBillKeywords}\n")
        # Hash tags
        yamlFile.write(f"hashTags: {thisBillHashTags}\n")
        # Bill and Session Information
        yamlFile.write(f"billNumber: {thisBillNumber}\n")
        yamlFile.write(f"billType: {legiscan.billCode[thisBillCode]}\n")
        yamlFile.write(f"legislativeBody: California Legislature\n")
        yamlFile.write(f"session: {bill['session']['session_tag']}\n")
        yamlFile.write(f"""topic: "{thisBillAlias1}: {bill['title']}"\n""")
        yamlFile.write(f"""title': "{thisBillAlias1}: {bill['description']}"\n""")
        yamlFile.write(f"""summary: "{aiBillsSummariesStored[thisPeriod][thisBillId]['summary']}"\n""")
        # Sponsors
        if not thisBillSponsors:
            yamlFile.write(f"sponsors: None\n")
            yamlFile.write(f"coSponsors: None\n")
            yamlFile.write(f"jointSponsors: None\n")
        else:
            yamlFile.write(f"sponsors: {', '.join(thisBillSponsors['Primary Sponsor'])}\n")
            yamlFile.write(f"coSponsors: {', '.join(thisBillSponsors['Co-Sponsor'])}\n")
            yamlFile.write(f"jointSponsors: {', '.join(thisBillSponsors['Joint Sponsor'])}\n")
        # Bill Status
        yamlFile.write(f"billStatus: {legiscan.statusType[bill['status']]}\n")
        if bill["session"]["sine_die"] == 0:
            yamlFile.write(f"sessionStatus: Active\n")
        elif bill["session"]["sine_die"] == 1:
            yamlFile.write(f"sessionStatus: Inactive\n")
        # Dates
        yamlFile.write(f"dateStatus: {bill['status_date']}\n")
        yamlFile.write(f"dateIntroduced: {bill['history'][0]['date']}\n")
        yamlFile.write(f"dateAssessed: {bill['history'][-1]['date']}\n")
        # Bill Actions
        yamlFile.write(f"lastAction: {thisBillAction}\n")
        yamlFile.write(f"chaptered: {chaptered}\n")
        yamlFile.write(f"chapterNo: {chapterNo}\n")
        yamlFile.write(f"chapterYear: {chapterYear}\n")
        # Legiscan link
        yamlFile.write(f"linkLegiscan: {bill['url']}\n")
        # California legislature bill links
        yamlFile.write(f"linkMain: {legisLinks['main']}\n")
        yamlFile.write(f"linkText: {legisLinks['text']}\n")
        yamlFile.write(f"linkVotes: {legisLinks['votes']}\n")
        yamlFile.write(f"linkHistory: {legisLinks['history']}\n")
        yamlFile.write(f"linkAnalysis: {legisLinks['analysis']}\n")
        yamlFile.write(f"linkTodaysLaw: {legisLinks['todaysLaw']}\n")
        yamlFile.write(f"linkCompare: {legisLinks['compare']}\n")
        yamlFile.write(f"linkStatus: {legisLinks['status']}\n")
        # Obsidian PDF link
        yamlFile.write(f"pdfLink: [[Documents/CA Legislative Bills/{thisPeriod}/{thisBillId}.pdf]]\n")
        # Legiscan IDs
        yamlFile.write(f"legiscanBillId: {bill['bill_id']}\n")
        yamlFile.write(f"legiscanBillHash: {bill['change_hash']}\n")
        yamlFile.write(f"legiscanSessionId: {bill['session_id']}\n")
        # Related (vector)
        yamlFile.write(f"related:\n")
        yamlFile.write(f"  - [[Artificial Intelligence]]\n")
        yamlFile.write(f"  - [[California Government]]\n")
        # Dates for Obsidian
        yamlFile.write(f"generated: \n")
        yamlFile.write(f"modified: \n")
        # Close the YAML file
        yamlFile.write(f"---\n")

In [141]:
aiBillYaml("2013-2014", "SB860")

In [316]:
def aiBillMarkdown(billPeriod, billId):
    """
    Function to create an md file for the AI bill
    """
    # Get the bill data from the AI bills dictionary
    bill = aiBills[billPeriod][billId]
    
    thisPeriod = billPeriod
    thisBillId = billId
    thisBillCode = f"{bill['body']}{bill['bill_type']}"
    thisBillNumber = bill['bill_number'].replace(f"{thisBillCode}", "")
    thisBillAlias1 = f"{thisBillCode}-{thisBillNumber}"
    thisBillAlias2 = f"{thisBillCode} {thisBillNumber}"
    thisBillKeywords = ", ".join(aiBillsSummariesStored[thisPeriod][thisBillId]["tags"])
    thisBillHashTags = ", ".join([f"#{tag}" for tag in aiBillsSummariesStored[thisPeriod][thisBillId]["tags"]])
    thisBillSponsors = legiscan.summarizeBillSponsors(aiBills[thisPeriod][thisBillId], output="dict")
    thisBillSponsorsMd = legiscan.summarizeBillSponsors(aiBills[thisPeriod][thisBillId], output="md")
    thisBillAction = bill['history'][-1]['action']
    if thisBillAction.startswith("Chaptered"):
        chaptered = True
        chapterNo = thisBillAction.removeprefix("Chaptered by Secretary of State. Chapter ").removesuffix(".")
        chapterYear = int(chapterNo.split(", Statutes of ")[1])
        chapterNo = int(chapterNo.split(", Statutes of ")[0])
    else:
        chaptered = False
        chapterYear = None
        chapterNo = None
    
    legisLinks = calpa.getCaLegisLinks(billPeriod, billId)
    thisNotesPath = os.path.join(prjDirs["pathScriptsMd"], "notes", thisPeriod, f"{thisBillId}.md")
    with open(thisNotesPath, 'r') as src:
        thisBillNotes = src.readlines()
        section = aiNotes = lcNotes = ""
        for i, line in enumerate(thisBillNotes):
            if line.startswith(f"## {thisBillId} AI Notes"):
                section = "AI"
                #print(f"Found AI Notes at line {i}")
            elif line.startswith(f"## {thisBillId} LC Notes"):
                section = "LC"
                #print(f"Found LC Notes at line {i}")
            if section == "AI":
                aiNotes += line
            elif section == "LC":
                lcNotes += line
    
    # Create the YAML file name
    mdFileName = f"{billId}.md"
    
    # Create the YAML file path
    mdFilePath = os.path.join(prjDirs["pathScriptsMd"], "AI", thisPeriod, mdFileName)
    
    # Create the YAML file
    with open(mdFilePath, 'w') as mdFile:
        # Begin writing the YAML file
        mdFile.write(f"---\n")
        # Aliases (vector)
        mdFile.write(f"aliases:\n")
        mdFile.write(f"  - {thisBillAlias1}\n")
        mdFile.write(f"  - {thisBillAlias2}\n")
        # Type (vector)
        mdFile.write(f"type:\n")
        mdFile.write(f"  - {legiscan.billCode[thisBillCode]}\n")
        mdFile.write(f"  - California Legislature Bill\n")
        mdFile.write(f"  - AI Legislation\n")
        # Tags (vector)
        mdFile.write(f"tags:\n")
        mdFile.write(f"  - Zotero\n")
        mdFile.write(f"  - california-legislature\n")
        mdFile.write(f"  - {bill['session']['session_title'].lower().replace(' ', '-')}\n")
        for tag in aiBillsSummariesStored[thisPeriod][thisBillId]["tags"]:
            mdFile.write(f"  - {tag}\n")
        # Keywords
        mdFile.write(f"keywords: {thisBillKeywords}\n")
        # Hash tags
        mdFile.write(f"hashTags: '{thisBillHashTags}'\n")
        # Bill and Session Information
        mdFile.write(f"billNumber: {thisBillNumber}\n")
        mdFile.write(f"billType: {legiscan.billCode[thisBillCode]}\n")
        mdFile.write(f"legislativeBody: California Legislature\n")
        mdFile.write(f"session: {bill['session']['session_tag']}\n")
        mdFile.write(f"""topic: "{thisBillAlias1}: {bill['title']}"\n""")
        mdFile.write(f"""title': "{thisBillAlias1}: {bill['description']}"\n""")
        mdFile.write(f"""summary: "{aiBillsSummariesStored[thisPeriod][thisBillId]['summary']}"\n""")
        # Sponsors
        if "Primary Sponsor" not in thisBillSponsors.keys():
            mdFile.write(f"sponsors: None\n")
        else:
            mdFile.write(f"sponsors: {', '.join(thisBillSponsors['Primary Sponsor'])}\n")
        if "Co-Sponsor" not in thisBillSponsors.keys():
            mdFile.write(f"coSponsors: None\n")
        else:
            mdFile.write(f"coSponsors: {', '.join(thisBillSponsors['Co-Sponsor'])}\n")
        if "Joint Sponsor" not in thisBillSponsors.keys():
            mdFile.write(f"jointSponsors: None\n")
        else:
            mdFile.write(f"jointSponsors: {', '.join(thisBillSponsors['Joint Sponsor'])}\n")
        # Bill Status
        mdFile.write(f"billStatus: {legiscan.statusType[bill['status']]}\n")
        if bill["session"]["sine_die"] == 0:
            mdFile.write(f"sessionStatus: Active\n")
        elif bill["session"]["sine_die"] == 1:
            mdFile.write(f"sessionStatus: Inactive\n")
        # Dates
        mdFile.write(f"dateStatus: {bill['status_date']}\n")
        mdFile.write(f"dateIntroduced: {bill['history'][0]['date']}\n")
        mdFile.write(f"dateAssessed: {bill['history'][-1]['date']}\n")
        # Bill Actions
        mdFile.write(f"lastAction: {thisBillAction}\n")
        mdFile.write(f"chaptered: {chaptered}\n")
        mdFile.write(f"chapterNo: {chapterNo}\n")
        mdFile.write(f"chapterYear: {chapterYear}\n")
        # Legiscan link
        mdFile.write(f"linkLegiscan: {bill['url']}\n")
        # California legislature bill links
        mdFile.write(f"linkMain: {legisLinks['main']}\n")
        mdFile.write(f"linkText: {legisLinks['text']}\n")
        mdFile.write(f"linkVotes: {legisLinks['votes']}\n")
        mdFile.write(f"linkHistory: {legisLinks['history']}\n")
        mdFile.write(f"linkAnalysis: {legisLinks['analysis']}\n")
        mdFile.write(f"linkTodaysLaw: {legisLinks['todaysLaw']}\n")
        mdFile.write(f"linkCompare: {legisLinks['compare']}\n")
        mdFile.write(f"linkStatus: {legisLinks['status']}\n")
        # Obsidian PDF link
        mdFile.write(f"pdfLink: '[[Documents/CA Legislative Bills/{thisPeriod}/{thisBillId}.pdf]]'\n")
        # Legiscan IDs
        mdFile.write(f"legiscanBillId: {bill['bill_id']}\n")
        mdFile.write(f"legiscanBillHash: {bill['change_hash']}\n")
        mdFile.write(f"legiscanSessionId: {bill['session_id']}\n")
        # Related (vector)
        mdFile.write(f"related:\n")
        mdFile.write(f"  - '[[Artificial Intelligence]]'\n")
        mdFile.write(f"  - '[[California Government]]'\n")
        # Dates for Obsidian
        mdFile.write(f"generated: \n")
        mdFile.write(f"modified: \n")
        # Close the YAML file
        mdFile.write(f"---\n")
        
        # Main Markdown Content
        mdFile.write("\n")
        # Session Title
        mdFile.write(f"## {thisBillAlias1}: {bill['title']}\n\n")
        
        # Metadata Info Box
        mdFile.write(f">[!legislative] **{thisBillId} Metadata**\n")
        mdFile.write(f">- **Bill Number**: {thisBillAlias1}\n")
        mdFile.write(f">- **Legislative Period**: {thisPeriod}\n")
        mdFile.write(f">- **Legislative Body**: California Legislature, {thisPeriod} {bill['session']['session_tag']}\n")
        mdFile.write(f">- **Bill Type**: {legiscan.billCode[thisBillCode]}\n")
        mdFile.write(f">- **Topic**: {thisBillAlias1}: {bill['title']}\n")
        mdFile.write(f">- **Title**: {thisBillAlias1}: {bill['description']}\n")
        mdFile.write(f">- **TL;DR Summary**: {aiBillsSummariesStored[thisPeriod][thisBillId]['summary']}\n")
        mdFile.write(f">- **Keywords**: {thisBillKeywords}\n")
        mdFile.write(f">- **Hash Tags**: {thisBillHashTags}\n")
        if "Primary Sponsor" not in thisBillSponsorsMd.keys():
            mdFile.write(f">- **Sponsor(s)**: None\n")
        else:
            mdFile.write(f">- **Sponsor(s)**: {thisBillSponsorsMd['Primary Sponsor']}\n")
        if "Co-Sponsor" not in thisBillSponsorsMd.keys():
            mdFile.write(f">- **Co-Sponsor(s)**: None\n")
        else:
            mdFile.write(f">- **Co-Sponsor(s)**: {thisBillSponsorsMd['Co-Sponsor']}\n")
        if "Joint Sponsor" not in thisBillSponsorsMd.keys():
            mdFile.write(f">- **Joint Sponsor(s)**: None\n")
        else:
            mdFile.write(f">- **Joint Sponsor(s)**: {thisBillSponsorsMd['Joint Sponsor']}\n")
        mdFile.write(f">- **Introduced Date**: {calpa.convertStrToDate(bill['history'][0]['date'])}\n")
        mdFile.write(f">- **Bill Status**: {legiscan.statusType[bill['status']]}\n")
        if bill["session"]["sine_die"] == 0:
            mdFile.write(f">- **Session Status**: Active\n")
        elif bill["session"]["sine_die"] == 1:
            mdFile.write(f">- **Session Status**: Inactive\n")
        mdFile.write(f">- **Status Date**: {calpa.convertStrToDate(bill['status_date'])}\n")
        mdFile.write(f">- **Last Action**: {thisBillAction}\n")
        mdFile.write(f">- **Last Action Date**: {calpa.convertStrToDate(bill['history'][-1]['date'])}\n")
        mdFile.write(f">- **Chaptered**: {chaptered}\n")
        mdFile.write(f">- **Chapter No**: {chapterNo}\n")
        mdFile.write(f">- **Chapter Year**: {chapterYear}\n")
        mdFile.write(f">- **LegiScan Bill ID**: {bill['bill_id']}\n")
        mdFile.write(f">- **LegiScan Bill Hash**: {bill['change_hash']}\n")
        mdFile.write(f">- **LegiScan Session ID**: {bill['session_id']}\n")
        mdFile.write(f">- **Bill Links**: ")
        mdFile.write(f"[LegiScan]({bill['url']}), ")
        mdFile.write(f"[State Main]({legisLinks['main']}), ")
        mdFile.write(f"[State Text]({legisLinks['text']}), ")
        mdFile.write(f"[State Votes]({legisLinks['votes']}), ")
        mdFile.write(f"[State History]({legisLinks['history']}), ")
        mdFile.write(f"[State Analysis]({legisLinks['analysis']}), ")
        mdFile.write(f"[State Today's Law]({legisLinks['todaysLaw']}), ")
        mdFile.write(f"[State Compare]({legisLinks['compare']}), ")
        mdFile.write(f"[State Status]({legisLinks['status']})\n")
        mdFile.write(f">- **Obsidian PDF Link**: [[Documents/CA Legislative Bills/{thisPeriod}/{thisBillId}.pdf]]\n")
        mdFile.write(f">- **Related**: [[Artificial Intelligence]], [[California Government]]\n\n")
        
        # Summary Info Box
        mdFile.write(f">[!tldr] **{thisBillId} TL;DR Summary**\n")
        mdFile.write(f"> {aiBillsSummariesStored[thisPeriod][thisBillId]['summary']}\n\n\n")
        
        # Write the bill Notes
        mdFile.write(f"{aiNotes}\n")  
        
        # Webpage Title
        mdFile.write(f"## State WebPage\n\n")
        mdFile.write(f"""<iframe src="{legisLinks['main']}" allow="fullscreen" allowfullscreen="" style="height: 100%;width:100%;aspect-ratio: 16/ 10;"</iframe>\n""")
        
        
        mdFile.write("\n")

In [317]:
aiBillMarkdown("2013-2014", "AB1465")
aiBillMarkdown("2013-2014", "SB836")
aiBillMarkdown("2013-2014", "SB860")

In [ ]:
<iframe src="https://leginfo.legislature.ca.gov/faces/billTextClient.xhtml?bill_id=202520260AB33" allow="fullscreen" allowfullscreen="" style="height:100%;width:100%; aspect-ratio: 16 / 10; "></iframe>


In [ ]:
# format date to MMM-DD-YYYY
def formatDate(date):
    return date.strftime("%b-%d-%Y")

In [198]:
# convert "2023-10-01" to date
def convertDate(dateString):
    return datetime.strptime(dateString, "%Y-%m-%d").date().strftime("%B %d, %Y")
convertDate("2023-10-01")

'October 01, 2023'

In [133]:
aiBills["2013-2014"]["SB860"]

{'bill_id': 581712,
 'change_hash': 'e38cd0ab53209fc7aaf182ffc8ba1f5b',
 'session_id': 993,
 'session': {'session_id': 993,
  'state_id': 5,
  'year_start': 2013,
  'year_end': 2014,
  'prefile': 0,
  'sine_die': 1,
  'prior': 1,
  'special': 0,
  'session_tag': 'Regular Session',
  'session_title': '2013-2014 Regular Session',
  'session_name': '2013-2014 Session'},
 'url': 'https://legiscan.com/CA/bill/SB860/2013',
 'state_link': 'http://www.leginfo.ca.gov/cgi-bin/postquery?bill_number=sb_860&sess=1314&house=S',
 'completed': 1,
 'status': 4,
 'status_date': '2014-06-20',
 'progress': [{'date': '2014-01-09', 'event': 1},
  {'date': '2014-01-23', 'event': 9},
  {'date': '2014-04-10', 'event': 2},
  {'date': '2014-05-08', 'event': 9},
  {'date': '2014-06-12', 'event': 9},
  {'date': '2014-06-15', 'event': 3},
  {'date': '2014-06-20', 'event': 8},
  {'date': '2014-06-20', 'event': 4}],
 'state': 'CA',
 'state_id': 5,
 'bill_number': 'SB860',
 'bill_type': 'B',
 'bill_type_id': '1',
 'bo

In [ ]:
webbrowser.open(aiBills["2013-2014"]["AB1465"]["url"], new=2, autoraise=True)

In [ ]:
webbrowser.open(aiBills["2013-2014"]["AB1465"]["state_link"], new=2, autoraise=True)

In [ ]:
aiBills["2013-2014"]["AB1465"]["progress"][-1]["date"]

In [ ]:
aiBills["2013-2014"]["AB1465"]["history"][-1]

In [44]:
legiscan.summarizeBillSponsors(aiBills["2013-2014"]["SB836"], output="dict")

{'Primary Sponsor': ['Ellen Corbett (D, SD10)'],
 'Co-Sponsor': ['Joel Anderson (R, SD38)',
  'Jim Beall (D, SD15)',
  'Marty Block (D, SD39)',
  'Anthony Cannella (R, SD12)',
  'Lou Correa (D, SD34)',
  'Noreen Evans (D, SD02)',
  'Cathleen Galgiani (D, SD05)',
  'Loni Hancock (D, SD09)',
  'Jerry Hill (D, SD13)',
  'Ricardo Lara (D, SD33)',
  'Ted Lieu (D, SD28)',
  'Carol Liu (D, SD25)',
  'Jim Nielsen (R, SD04)',
  'Richard Roth (D, SD31)',
  'Lois Wolk (D, SD03)'],
 'Joint Sponsor': ['Nancy Skinner (D, SD09)', 'Marie Waldron (R, AD75)']}

In [68]:
test = legiscan.summarizeBillSponsors(aiBills["2013-2014"]["SB860"], output="dict")

In [71]:
# check if test is empty
if not test:
    print("test is empty")

test is empty


In [45]:
test = legiscan.summarizeBillSponsors(aiBills["2013-2014"]["SB836"], output="dict")

In [57]:
', '.join(test['Primary Sponsor'])

'Ellen Corbett (D, SD10)'

In [58]:
', '.join(test['Co-Sponsor'])

'Joel Anderson (R, SD38), Jim Beall (D, SD15), Marty Block (D, SD39), Anthony Cannella (R, SD12), Lou Correa (D, SD34), Noreen Evans (D, SD02), Cathleen Galgiani (D, SD05), Loni Hancock (D, SD09), Jerry Hill (D, SD13), Ricardo Lara (D, SD33), Ted Lieu (D, SD28), Carol Liu (D, SD25), Jim Nielsen (R, SD04), Richard Roth (D, SD31), Lois Wolk (D, SD03)'

In [59]:
', '.join(test['Joint Sponsor'])

'Nancy Skinner (D, SD09), Marie Waldron (R, AD75)'

In [178]:
test=legiscan.summarizeBillSponsors(aiBills["2013-2014"]["AB1465"], output="md")

In [180]:
if not test:
    print("test is empty")
else:
    if not test["Primary Sponsor"]:
        print("test['Primary Sponsor'] is empty")
    if not test["Co-Sponsor"]:
        print("test['Co-Sponsor'] is empty")
    if not test["Joint Sponsor"]:
        print("test['Joint Sponsor'] is empty")

KeyError: 'Co-Sponsor'

In [42]:
legiscan.summarizeBillSponsors(aiBills["2013-2014"]["SB836"], output="md")

- Primary Sponsor(s): [Ellen Corbett (D, SD10)](https://ballotpedia.org/Ellen_Corbett)
- Co-Sponsor(s): [Joel Anderson (R, SD38)](https://ballotpedia.org/Joel_Anderson_(California)), [Jim Beall (D, SD15)](https://ballotpedia.org/James_Beall_Jr.), [Marty Block (D, SD39)](https://ballotpedia.org/Martin_Block), [Anthony Cannella (R, SD12)](https://ballotpedia.org/Anthony_Cannella), [Lou Correa (D, SD34)](https://ballotpedia.org/Lou_Correa), [Noreen Evans (D, SD02)](https://ballotpedia.org/Noreen_Evans), [Cathleen Galgiani (D, SD05)](https://ballotpedia.org/Cathleen_Galgiani), [Loni Hancock (D, SD09)](https://ballotpedia.org/Loni_Hancock), [Jerry Hill (D, SD13)](https://ballotpedia.org/Gerald_Hill), [Ricardo Lara (D, SD33)](https://ballotpedia.org/Ricardo_Lara), [Ted Lieu (D, SD28)](https://ballotpedia.org/Ted_Lieu), [Carol Liu (D, SD25)](https://ballotpedia.org/Carol_Liu), [Jim Nielsen (R, SD04)](https://ballotpedia.org/Jim_Nielsen), [Richard Roth (D, SD31)](https://ballotpedia.org/Richar

{'Primary Sponsor': '[Ellen Corbett (D, SD10)](https://ballotpedia.org/Ellen_Corbett)',
 'Co-Sponsor': '[Joel Anderson (R, SD38)](https://ballotpedia.org/Joel_Anderson_(California)), [Jim Beall (D, SD15)](https://ballotpedia.org/James_Beall_Jr.), [Marty Block (D, SD39)](https://ballotpedia.org/Martin_Block), [Anthony Cannella (R, SD12)](https://ballotpedia.org/Anthony_Cannella), [Lou Correa (D, SD34)](https://ballotpedia.org/Lou_Correa), [Noreen Evans (D, SD02)](https://ballotpedia.org/Noreen_Evans), [Cathleen Galgiani (D, SD05)](https://ballotpedia.org/Cathleen_Galgiani), [Loni Hancock (D, SD09)](https://ballotpedia.org/Loni_Hancock), [Jerry Hill (D, SD13)](https://ballotpedia.org/Gerald_Hill), [Ricardo Lara (D, SD33)](https://ballotpedia.org/Ricardo_Lara), [Ted Lieu (D, SD28)](https://ballotpedia.org/Ted_Lieu), [Carol Liu (D, SD25)](https://ballotpedia.org/Carol_Liu), [Jim Nielsen (R, SD04)](https://ballotpedia.org/Jim_Nielsen), [Richard Roth (D, SD31)](https://ballotpedia.org/Richar

- Primary Sponsor(s): [Ellen Corbett (D, SD10)](https://ballotpedia.org/Ellen_Corbett)
- Co-Sponsor(s): [Joel Anderson (R, SD38)](https://ballotpedia.org/Joel_Anderson_(California)), [Jim Beall (D, SD15)](https://ballotpedia.org/James_Beall_Jr.), [Marty Block (D, SD39)](https://ballotpedia.org/Martin_Block), [Anthony Cannella (R, SD12)](https://ballotpedia.org/Anthony_Cannella), [Lou Correa (D, SD34)](https://ballotpedia.org/Lou_Correa), [Noreen Evans (D, SD02)](https://ballotpedia.org/Noreen_Evans), [Cathleen Galgiani (D, SD05)](https://ballotpedia.org/Cathleen_Galgiani), [Loni Hancock (D, SD09)](https://ballotpedia.org/Loni_Hancock), [Jerry Hill (D, SD13)](https://ballotpedia.org/Gerald_Hill), [Ricardo Lara (D, SD33)](https://ballotpedia.org/Ricardo_Lara), [Ted Lieu (D, SD28)](https://ballotpedia.org/Ted_Lieu), [Carol Liu (D, SD25)](https://ballotpedia.org/Carol_Liu), [Jim Nielsen (R, SD04)](https://ballotpedia.org/Jim_Nielsen), [Richard Roth (D, SD31)](https://ballotpedia.org/Richard_Roth), [Lois Wolk (D, SD03)](https://ballotpedia.org/Lois_Wolk)
- Joint Sponsor(s): [Nancy Skinner (D, SD09)](https://ballotpedia.org/Nancy_Skinner_(California)), [Marie Waldron (R, AD75)](https://ballotpedia.org/Marie_Waldron)

In [167]:
test = legiscan.summarizeBillSponsors(aiBills["2013-2014"]["SB836"], output="md")

In [168]:
test["Primary Sponsor"]

'[Ellen Corbett (D, SD10)](https://ballotpedia.org/Ellen_Corbett)'

In [ ]:
myBill = aiBills["2013-2014"]["SB836"]

In [ ]:
myBillDocId = myBill["texts"][-1]["doc_id"]

In [ ]:
myBillText =legiscan.getBillText(myBillDocId)

In [229]:
samplePath = r"C:\Users\ktale\Knowledge Management\Policy and Governance\Legislation\Labor Bills\2025-2026\AB-21.md"

In [238]:

analyzer = MarkdownAnalyzer(samplePath)

headers = analyzer.identify_headers()
paragraphs = analyzer.identify_paragraphs()
blockquotes = analyzer.identify_blockquotes()
links = analyzer.identify_links()
codeBlocks = analyzer.identify_code_blocks()

In [ ]:
analysis = analyzer.analyse()

print(analysis)

{'headers': 7, 'paragraphs': 10, 'blockquotes': 5, 'code_blocks': 0, 'ordered_lists': 0, 'unordered_lists': 0, 'tables': 0, 'html_blocks': 1, 'html_inline_count': 0, 'words': 799, 'characters': 7241}


In [231]:
headers

{'Header': [{'line': 58,
   'level': 2,
   'text': 'AB-21 Taxpayer Protection Act of 2025'},
  {'line': 120, 'level': 2, 'text': 'Webpage'},
  {'line': 125, 'level': 2, 'text': 'Self Notes'},
  {'line': 131, 'level': 2, 'text': 'OCEA Legislative Committee Notes'},
  {'line': 132, 'level': 3, 'text': 'OCEA LC Meeting: 25/02'},
  {'line': 138, 'level': 3, 'text': 'OCEA LC Meeting: 25/03'},
  {'line': 144, 'level': 2, 'text': 'Reading notes'}]}

In [232]:
paragraphs

{'Paragraph': ['The California Constitution also provides that all taxes imposed by a local government are either general taxes or special taxes, as defined, and requires that taxes imposed, extended, or increased by a local government be submitted to the electorate and approved by a majority vote, in the case of general taxes, or a 2/3 vote, in the case of special taxes.\nExisting law imposes specified requirements on state and local ballots, including, among other things, on the contents of the ballot label, ballot title, and summary.\nThis bill would declare the intent of the Legislature to enact a constitutional amendment to limit the ability of state and local governments to raise taxes, restore a 2/3 vote requirement on local special tax increases, impose voter approval requirements on specific categories of new taxes, and regulate the titles on state and local ballot measures relating to tax increases.',
  '%% begin notes %%',
  '%% end notes %%',
  '**Discussion:**',
  '**Dispo

In [233]:
links

{'Text link': [{'line': 60,
   'text': 'Zotero URI',
   'url': 'https://zotero.org/groups/5920243/items/QJI3JVD8'},
  {'line': 60,
   'text': 'PDF',
   'url': 'zotero://select/groups/5920243/items/3YVM7HR6'},
  {'line': 60,
   'text': 'Full Text',
   'url': 'https://leginfo.legislature.ca.gov/faces/billTextClient.xhtml?bill_id=202520260AB21'},
  {'line': 60,
   'text': 'History',
   'url': 'https://leginfo.legislature.ca.gov/faces/billHistoryClient.xhtml?bill_id=202520260AB21'},
  {'line': 60,
   'text': 'Status',
   'url': 'https://leginfo.legislature.ca.gov/faces/billStatusClient.xhtml?bill_id=202520260AB21'},
  {'line': 60,
   'text': 'Sponsor',
   'url': 'https://ballotpedia.org/Carl_DeMaio'},
  {'line': 108,
   'text': 'https://leginfo.legislature.ca.gov/faces/billTextClient.xhtml?bill_id=202520260AB21',
   'url': 'https://leginfo.legislature.ca.gov/faces/billTextClient.xhtml?bill_id=202520260AB21'}]}

In [ ]:
import io
import pypandoc
import panflute

def action(elem, doc):
    if isinstance(elem, panflute.Image):
        doc.images.append(elem)
    elif isinstance(elem, panflute.Link):
        doc.links.append(elem)

if __name__ == '__main__':
    data = pypandoc.convert_file('example.md', 'json')
    doc = panflute.load(io.StringIO(data))
    doc.images = []
    doc.links = []
    doc = panflute.run_filter(action, prepare=prepare, doc=doc)

    print("\nList of image URLs:")
    for image in doc.images:
        print(image.url)

In [249]:
# Directory for the markdown notes
mdNotesPath = os.path.join(prjDirs["pathScriptsMd"], "notes")

for key in aiBills.keys():
    # check if a directory exists for the bill period
    if not os.path.exists(os.path.join(mdNotesPath, key)):
        # create the directory
        os.makedirs(os.path.join(mdNotesPath, key))
    for billId in aiBills[key].keys():
        # Check if the bill has a markdown file
        if not os.path.exists(os.path.join(mdNotesPath, key, f"{billId}.md")):
            # Create the markdown file
            with open(os.path.join(mdNotesPath, key, f"{billId}.md"), 'w') as mdFile:
                # Write the bill information to the markdown file
                mdFile.write(f"## {billId} AI Notes\n\n")
        # othewise replace the file with the new one
        else:
            # Replace the markdown file with the new one
            with open(os.path.join(mdNotesPath, key, f"{billId}.md"), 'w') as mdFile:
                # Write the bill information to the markdown file
                mdFile.write(f"## {billId} AI Notes\n\n")


In [253]:
testPath = os.path.join(prjDirs["pathScriptsMd"], "notes", "2013-2014", "AB1465.md")

with open(testPath, 'r') as src:
        testNotes = src.read()


In [259]:
testNotes

'## AB1465 AI Notes\n\nTest AI notes for AB1465. \n\n\n## AB1465 LC Notes\n\n### OCEA-LC-25-02\n\nNotes for February 2025 OCEA Legislative Committee meeting.\n- Notes here.\n'

In [262]:
# Read the testPath markdown file, and find the section "## AI Notes"
testNotes.split("\n## AB1465 LC Notes")[1]

'\n\n### OCEA-LC-25-02\n\nNotes for February 2025 OCEA Legislative Committee meeting.\n- Notes here.\n'

In [309]:
with open(testPath, 'r') as src:
    lines = src.readlines()
    section = aiNotes = lcNotes = ""
    for i, line in enumerate(lines):
        if line.startswith("## AB1465 AI Notes"):
            section = "AI"
            #print(f"Found AI Notes at line {i}")
        elif line.startswith("## AB1465 LC Notes"):
            section = "LC"
            #print(f"Found LC Notes at line {i}")
        if section == "AI":
            aiNotes += line
        elif section == "LC":
            lcNotes += line
